In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import pickle

import preprocessing as pp
import models as md

from scipy import stats, integrate, optimize

In [4]:
df = pd.read_csv("../data/wholetime.csv")

df = df.dropna()

# データの分割
df_p1 = df[(df['started_at'] >= '2020-03-01') & (df['started_at'] < '2020-06-01')].dropna()
df_p2 = df[(df['started_at'] >= '2020-06-01') & (df['started_at'] < '2020-11-01')].dropna()
df_p3 = df[(df['started_at'] >= '2020-12-01') & (df['started_at'] < '2021-03-01')].dropna()
df_p4 = df[(df['started_at'] >= '2021-03-01') & (df['started_at'] < '2023-10-01')].dropna()
df_p4_1 = df[(df['started_at'] >= '2021-03-01') & (df['started_at'] < '2022-01-01')].dropna()
df_p4_2 = df[(df['started_at'] >= '2022-01-01') & (df['started_at'] < '2023-10-01')].dropna()
df_left = pd.concat([df_p1, df_p2, df_p3, df_p4_1], axis=0)
df_right = pd.concat([df_p4_2], axis=0)

/var/folders/pm/4cgtm7w90fbgczj718s4vvk40000gn/T/ipykernel_33822/1915566129.py:1: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/wholetime.csv")


In [5]:
def return_nm(df):
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    # started_atで昇順にソート
    df = df.sort_values(by='started_at')
    current_timestamp = df['started_at'].min()
    end_timestamp = df["started_at"].max()
    interval = timedelta(days=1)

    node_list = []
    edge_list = []
    timestamp = []

    while current_timestamp <= end_timestamp:
        timestamp.append(current_timestamp)
        next_timestamp = current_timestamp + interval
        df_tmp = df[(df['started_at'] >= current_timestamp) & (df['started_at'] < next_timestamp)]

        # node_listにはユニークなstart_station_idを追加
        node_list.append(len(df_tmp['start_station_id'].unique()))

        # edge_listにはdf_tmpの行数を追加
        edge_list.append(len(df_tmp))

        current_timestamp = next_timestamp

    # node_listとedge_listで0の値を削除
    node_list = np.array(node_list)
    edge_list = np.array(edge_list)
    timestamp = np.array(timestamp)
    active_node_list = node_list[node_list != 0]
    edge_list = edge_list[edge_list != 0]
    timestamp = timestamp[:len(node_list)]
    
    return active_node_list, edge_list, timestamp

In [6]:
node_left, edge_left, timestamp_left = return_nm(df_left)
node_right, edge_right, timestamp_right = return_nm(df_right)

In [7]:
from scipy import stats, special, optimize, integrate

In [10]:
def model_beta(x, alpha, beta):
    Np = 744
    f = lambda a: ((a**(alpha-1)) * ((1-a)**(beta-1)) / special.beta(alpha, beta) ) * (1 - (((alpha + beta) / alpha) * 2*a*x[0])/(Np**2-Np))**(Np - 1)
    ff, _ = integrate.quad(f, 0, 1)  # 積分の値と推定誤差を別々に受け取る
    func = Np * (1 - ff) 
    return func

In [11]:
print("+------------------+")
print("| Estimating       |")
print("+------------------+")

params, _ = optimize.curve_fit(model_beta, xdata=edge_right, ydata=node_right, p0=[0.5, 0.6])

+------------------+
| Estimating       |
+------------------+


/var/folders/pm/4cgtm7w90fbgczj718s4vvk40000gn/T/ipykernel_33822/3295451428.py:3: RuntimeWarning: overflow encountered in scalar power
  f = lambda a: ((a**(alpha-1)) * ((1-a)**(beta-1)) / special.beta(alpha, beta) ) * (1 - (((alpha + beta) / alpha) * 2*a*x[0])/(Np**2-Np))**(Np - 1)
/var/folders/pm/4cgtm7w90fbgczj718s4vvk40000gn/T/ipykernel_33822/3295451428.py:4: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  ff, _ = integrate.quad(f, 0, 1)  # 積分の値と推定誤差を別々に受け取る
/var/folders/pm/4cgtm7w90fbgczj718s4vvk40000gn/T/ipykernel_33822/3295451428.py:4: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  ff, _ = integrate.quad(f, 0, 1)  # 積分の値と推定誤差を別々に受け取る
